In [1]:
#import relevant libraries
#Load modules
import os

# if os.path.exists('D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\MBON_Chrimson_Farhan\\')==True:
#     os.chdir('D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\MBON_Chrimson_Farhan\\') #Put the location you want this file to be in

import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sbs
import dabest
from datetime import datetime
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from tqdm import tqdm
from time import sleep
import plotly.graph_objects as go
from scipy.signal import find_peaks
from numpy import genfromtxt
from scipy import fftpack
import scipy.stats as stats
import scipy 
from struct import *
import math
import osar
import sys
import glob
import scipy as sp
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import re
import warnings
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
#warnings.simplefilter(action="default", category=RuntimeWarning)

# Computer type

In [2]:
#typeo = "C:\\Users\\user\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\"
#typeo = "C:\\Users\\lnico\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\"
typeo = "D:\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\"
genot = "ACR"
typeob = typeo + genot + "\\"

# Run if you have new data

In [4]:
#nicole's data (only run if no comp set)
root = typeob
startdir = os.listdir(root)
baseDirectory=[]
basegenotype=[]
for n,sd in enumerate(startdir):
    dir1 = os.path.join(root, os.listdir(root)[n])
    baseDirectory.append(dir1)
    basegenotype.append(re.search(r'^.*?(?= x )', (os.path.split(dir1)[1])).group(0))
    #basegenotype.append(re.search(r'^.*?(?=-Gal4)', (os.path.split(dir1)[1])).group(0))
        
        
dfs=[]
for loc,name in zip(baseDirectory,basegenotype):
    dfs.append(osar.osar(loc,driver = name).results)

Creating borders for each fly...
Done.

Processing CSV 35 of 35
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 71 of 71
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 56 of 56
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 57 of 57
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 52 of 52
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 65 of 65
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 84 of 84
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 68 of 68
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 79 of 79
Summarising results for all flies...
All done.
C

In [5]:
df=pd.concat(dfs)
filepath = typeo + genot + "compilation_full.csv"
df.to_csv(filepath)
#df.to_csv('D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\ACRcompilation_full.csv')

In [10]:
cols = ['Genotype', 'pi_smoothed_Pattern 01','reversal_light_pi', 'traversal_light_pi','light_attraction_index_baseline_corrected','log2_speed_ratio']
lst = []
for loc,name in zip(baseDirectory,basegenotype):
    
    df_osar = osar.osar(loc,driver = name).results

    df1 = df_osar[(df_osar['light_intensity']=='Full')].loc[:,['light_intensity','status','pi_smoothed_Pattern 01', 'reversal_light_pi', 'traversal_light_pi', 'light_attraction_index_baseline_corrected', 'log2_speed_ratio']].reset_index()
    test = df1.loc[:,['status','log2_speed_ratio']]
    test.replace([np.inf, -np.inf], np.nan, inplace=True)
    test.dropna(inplace=True)
    lst.append([name, dabest.load(df1, idx=("Sibling", "Offspring"),x="status", y="pi_smoothed_Pattern 01").mean_diff.results.difference[0],
               dabest.load(df1, idx=("Sibling", "Offspring"),x="status", y="reversal_light_pi").mean_diff.results.difference[0],
               dabest.load(df1, idx=("Sibling", "Offspring"),x="status", y="traversal_light_pi").mean_diff.results.difference[0],
               dabest.load(df1, idx=("Sibling", "Offspring"),x="status", y="light_attraction_index_baseline_corrected").mean_diff.results.difference[0],
               dabest.load(test, idx=("Sibling", "Offspring"),x="status", y="log2_speed_ratio").mean_diff.results.difference[0]])
df1 = pd.DataFrame(lst, columns=cols)
df1.to_csv(typeo + 'deltas_ACRfull.csv')

Creating borders for each fly...
Done.

Processing CSV 71 of 71
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 56 of 56
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 57 of 57
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 52 of 52
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 65 of 65
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 84 of 84
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 68 of 68
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 79 of 79
Summarising results for all flies...
All done.
Creating borders for each fly...
Done.

Processing CSV 63 of 63
Summarising results for all flies...
All done.
C

c:\Users\user\anaconda3\lib\site-packages\dabest\_classes.py:2042: UserWarning: The lower limit of the interval was in the bottom 10 values. The result should be considered unstable.
  warnings.warn(err_temp.substitute(lim_type="lower",


# Run if you have old data

In [14]:
filepath = typeo + genot + "compilation_full.csv"
df2= pd.read_csv(filepath)
df1 = pd.read_csv(typeo + 'deltas_' + genot + 'full.csv')
basegenotype = df2['driver'].unique()

In [15]:
def meandiffchart(dfi, basegenotype, parameter):
    
    dfr1 = pd.DataFrame()
    for names in basegenotype:
        dfrr = pd.DataFrame()
        filter1 = dfi[(dfi['MBON'] == names) & (dfi['status'] == 'Offspring')].copy().reset_index(drop=True)
        filter2 = dfi[(dfi['MBON'] == names) & (dfi['status'] == 'Sibling')].copy().reset_index(drop=True)

        dfrr[names + " Cntrl" ] = filter2[parameter]
        dfrr[names] = filter1[parameter]
        dfr1 = pd.concat([dfr1, dfrr], axis = 1)
    l = dfr1.columns.tolist()
    titlelst = list(zip(l,l[1:]))[::2]  
    test = dabest.load(dfr1, idx=titlelst, ci=90)
    tab1 = test.mean_diff.statistical_tests
    tab1['difference'] = round(tab1['difference'],3)
    tab = pd.concat([tab1['test'], tab1['difference']], axis=1)
    tab.rename(columns = {'test':'MBON', 'difference':parameter}, inplace = True)

    return tab

In [16]:
import scikits.bootstrap as skb
from osar.plot_helpers.plot_helpers import r2_and_slope

#dfi=df2[(df2['light_intensity']=='Half') | (df2['light_intensity']=='Full')].copy()
dfi=df2[(df2['light_intensity']=='Full')].copy()
dfi.replace([np.inf, -np.inf], np.nan, inplace=True)
dfi.rename(columns={'driver': 'MBON'}, inplace = True)

parameter = 'pi_smoothed_Pattern 01'
#parameter = 'log2_speed_ratio_Pattern 01'

if parameter == 'log2_speed_ratio_Pattern 01':
    yaxi = 'Log2 Speed Ratio'
else:
    yaxi = 'PI'

In [17]:
dfr1 = pd.DataFrame()
for names in basegenotype:
    dfrr = pd.DataFrame()
    filter1 = dfi[(dfi['MBON'] == names) & (dfi['status'] == 'Offspring')].copy().reset_index(drop=True)
    filter2 = dfi[(dfi['MBON'] == names) & (dfi['status'] == 'Sibling')].copy().reset_index(drop=True)
    
    dfrr[names + " Cntrl" ] = filter2[parameter]
    dfrr[names] = filter1[parameter]
    dfr1 = pd.concat([dfr1, dfrr], axis = 1)
    
l = dfr1.columns.tolist()
titlelst = list(zip(l,l[1:]))[::2]    

In [19]:
#massplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score,r2_score
import plotly.express as px

dfi["abs_pi_smoothed_bs"] = abs(dfi["pi_smoothed_baseline_corrected"])
dfi["abs_pi_smoothed_nbs"] = abs(dfi["pi_smoothed_not_baseline_corrected"])
dfi["abs_pi_smoothed_01"] = abs(dfi["pi_smoothed_Pattern 01"])
dfi["abs_pi_smoothed_10"] = abs(dfi["pi_smoothed_Pattern 10"])

fig = make_subplots(rows=12, cols=4, row_heights=12*[0.125],shared_xaxes=True,vertical_spacing=0.02, horizontal_spacing = 0.05)


for name, n,color in zip(basegenotype, range(1, len(basegenotype)+1),px.colors.qualitative.Dark24):
    
    gen = dfi[dfi['MBON'] == name]
    gen.replace(np. nan,'',regex=True)
    
    fig.append_trace(go.Scattergl(x =(gen['log2_speed_ratio']),
    y=gen['abs_pi_smoothed_01'],
    mode='markers', marker = dict(size=4, opacity = 0.5, color=color),
    name='abs PI vs Speed ratio'
), row=n, col=1)
    #regression
    gengen=pd.DataFrame()
    gengen = gen.loc[:,['log2_speed_ratio', 'abs_pi_smoothed_01']]
    gengen.dropna(inplace=True)
    reg = LinearRegression().fit(np.vstack(gengen['log2_speed_ratio']), gengen['abs_pi_smoothed_01'])
    gengen['bestfit_1'] = reg.predict(np.vstack(gengen['log2_speed_ratio'])) 
    r2_1 = reg.score(gengen['log2_speed_ratio'].values.reshape(-1, 1), gengen['abs_pi_smoothed_01'].values)
    fig.add_trace(go.Scatter(name='line of best fit', x=gengen['log2_speed_ratio'], y=gengen['bestfit_1'], mode='lines'), row=n, col=1)
    fig.update_traces(line_color="#000000", selector=dict(type='scatter'), row=n, col=1)
    fig.add_annotation(x=0.70, y=0.9,
            text='R\u00b2='+str("{:.3f}".format(r2_1)), xref="paper", yref="paper",
            showarrow=False,row=n,col=1)

        
    
    fig.append_trace(go.Scattergl(x =(gen['log2_speed_ratio']),
    y=gen['pi_smoothed_Pattern 01'],
    mode='markers',
    name='PI vs Speed ratio',marker = dict(size=4, opacity = 0.5, color=color),
), row=n, col=2)
    #regression
    gengen=pd.DataFrame()
    gengen = gen.loc[:,['log2_speed_ratio', 'pi_smoothed_Pattern 01']]
    gengen.dropna(inplace=True)
    reg = LinearRegression().fit(np.vstack(gengen['log2_speed_ratio']), gengen['pi_smoothed_Pattern 01'])
    gengen['bestfit_1'] = reg.predict(np.vstack(gengen['log2_speed_ratio']))   
    r2_2 = reg.score(gengen['log2_speed_ratio'].values.reshape(-1, 1), gengen['pi_smoothed_Pattern 01'].values)
    fig.add_trace(go.Scatter(name='line of best fit', x=gengen['log2_speed_ratio'], y=gengen['bestfit_1'], mode='lines'), row=n, col=2)
    fig.update_traces(line_color="#000000", selector=dict(type='scatter'), row=n, col=2)
    fig.add_annotation(x=0.70, y=0.8,
        text='R\u00b2='+str("{:.3f}".format(r2_2)),
        showarrow=False,xref="paper", yref="paper",row=n,col=2)                 
                     

    fig.append_trace(go.Scattergl(x =(gen['reversal_light_pi']),
    y=gen['pi_smoothed_Pattern 01'],
    mode='markers',
    name='PI vs reversal PI',marker = dict(size=4, opacity = 0.5, color=color),
), row=n, col=3)
    #regression
    gengen=pd.DataFrame()
    gengen = gen.loc[:,['reversal_light_pi', 'pi_smoothed_Pattern 01']]
    gengen.dropna(inplace=True)
    reg = LinearRegression().fit(np.vstack(gengen['reversal_light_pi']), gengen['pi_smoothed_Pattern 01'])
    gengen['bestfit_1'] = reg.predict(np.vstack(gengen['reversal_light_pi']))
    r2_3 = reg.score(gengen['reversal_light_pi'].values.reshape(-1, 1), gengen['pi_smoothed_Pattern 01'].values)
    fig.add_trace(go.Scatter(name='line of best fit', x=gengen['reversal_light_pi'], y=gengen['bestfit_1'], mode='lines'), row=n, col=3)
    fig.update_traces(line_color="#000000", selector=dict(type='scatter'), row=n, col=3)
    fig.add_annotation(x=0.70, y=0.8,
        text='R\u00b2='+str("{:.3f}".format(r2_3)),
        showarrow=False,xref="paper", yref="paper",row=n,col=3)

    
    
    fig.append_trace(go.Scattergl(x =(gen['traversal_light_pi']),
    y=gen['pi_smoothed_Pattern 01'],
    mode='markers',
    name='PI vs reversal PI',marker = dict(size=4, opacity = 0.5, color=color),
), row=n, col=4)    
    #regression
    gengen=pd.DataFrame()
    gengen = gen.loc[:,['traversal_light_pi', 'pi_smoothed_Pattern 01']]
    gengen.dropna(inplace=True)    
    reg = LinearRegression().fit(np.vstack(gengen['traversal_light_pi']), gengen['pi_smoothed_Pattern 01'])
    gengen['bestfit_1'] = reg.predict(np.vstack(gengen['traversal_light_pi']))
    r2_4 = reg.score(gengen['traversal_light_pi'].values.reshape(-1, 1), gengen['pi_smoothed_Pattern 01'].values)    
    fig.add_trace(go.Scatter(name='line of best fit', x=gengen['traversal_light_pi'], y=gengen['bestfit_1'], mode='lines'), row=n, col=4)
    fig.update_traces(line_color="#000000", selector=dict(type='scatter'), row=n, col=4)
    fig.add_annotation(x=0.70, y=0.8,
    text='R\u00b2='+str("{:.3f}".format(r2_4)),
    showarrow=False,xref="paper", yref="paper",row=n,col=4)
    

#x axes
fig.update_xaxes(title_text = "Speed ratio",row=12, col=1)
fig.update_xaxes(title_text = "Speed ratio",row=12, col=2)
fig.update_xaxes(title_text="Reversal PI",row=12, col=3)
fig.update_xaxes(title_text="Traversal PI",row=12, col=4)

for d in range(1,5):
    for c in range(1, len(basegenotype)+1):        
        fig.update_xaxes(range=[-1.1,1.1],tickvals=np.arange(-1,1.1,0.5),row=c, col=d)
    
#y axes
for c in range(1, len(basegenotype)+1):
    fig.add_annotation(x=-1.1, y=0.5, text='<b>|PI|</b>', showarrow=False,xshift =-50, yshift=0,row=c,col=1)
    fig.update_yaxes(range=[-0.1,1.1],tickvals=np.arange(0,1.1,0.5),row=c, col=1)


for d in range(2,5):
    for c in range(1, len(basegenotype)+1):        
        fig.update_yaxes(range=[-1.1,1.1],tickvals=np.arange(-1,1.1,0.5),row=c, col=d)
    
fig.add_annotation(x=-1.1, y=0, text='<b>PI</b>', showarrow=False,xshift =-35, yshift=0,row=1,col=2)


for name, n in zip(basegenotype, range(1, len(basegenotype)+1)):
    fig.add_annotation(x=-1.1, y=0.5, text= name, textangle =-90, showarrow=False,xshift =-80, yshift=0,row=n,col=1)

fig.update_layout(height=1800, width=1600, hovermode='x unified', showlegend=False,font=dict(family="ibm plex sans",size=18,), margin=dict(l=120, r=20, t=20, b=20))

#fig.write_image("D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\Images\\Nicole's figures\\Chrimson\\Total compilation of PI_smoothed_Pattern 01.svg")
fig.show()

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

#df10 = pd.read_csv('D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\deltas_Chrimson2.csv')
df10 = df1.copy()
df10.drop('Unnamed: 0', axis=1, inplace=True) 
df2=df10.copy()
df2.loc[len(df2)] = ['test', 1, 1,1,1,1]
df2.loc[len(df2)] = ['test', -1,-1,-1,-1,-1]

basegenotype = df2['Genotype'].unique()

def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())
for col in df2.columns[1:]:
    df2.loc[((df2[col]>1)), [col]] = 1
    df2.loc[((df2[col]<-1)), [col]] = -1
    df2[col] = min_max_scaling(df2[col])

rdbu = cm.get_cmap('RdBu_r', 100)
pic, revc, travc, latc,spdc=[],[],[],[],[]
for n in range(0,len(df2)):
    pic.append('rgb(' + str(rdbu(round(df2.iloc[n,1]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,1]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,1]*100))[2]*255)+')')
    revc.append('rgb(' + str(rdbu(round(df2.iloc[n,2]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,2]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,2]*100))[2]*255)+')')
    travc.append('rgb(' + str(rdbu(round(df2.iloc[n,3]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,3]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,3]*100))[2]*255)+')')
    latc.append('rgb(' + str(rdbu(round(df2.iloc[n,4]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,4]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,4]*100))[2]*255)+')')
    spdc.append('rgb(' + str(rdbu(round(df2.iloc[n,5]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,5]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,5]*100))[2]*255)+')')

pi, rev, trav, lat,spd=[],[],[],[],[]
for n in range(0,len(df10)):
    pi.append(str("{:.2f}".format(df10.iloc[n,1])))
    rev.append(str("{:.2f}".format(df10.iloc[n,2])))
    trav.append(str("{:.2f}".format(df10.iloc[n,3])))
    lat.append(str("{:.2f}".format(df10.iloc[n,4])))
    spd.append(str("{:.2f}".format(df10.iloc[n,5])))

#root = "D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\Chrimson2\\"
root = "D:\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\Chrimson2\\"

startdir = os.listdir(root)
baseDirectory=[]
basegenotype=[]
for n,sd in enumerate(startdir):
    dir1 = os.path.join(root, os.listdir(root)[n])
    baseDirectory.append(dir1)
    basegenotype.append(re.search(r'^.*?(?= x)', (os.path.split(dir1)[1])).group(0))
    
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Genotype</b>', '<b>Δ pi</b>','<b>Δ Reversal pi</b>', '<b>Δ Traversal pi</b>','<b>Δ Light Attraction index</b>','<b>Δ Speed Ratio</b>'],
    line_color='darkslategray', fill_color='white',
    align='center',font=dict(color='black', size=14)
  ),
  cells=dict(
    values=[basegenotype, pi, rev, trav, lat,spd],
    line_color=['darkslategray'],
    fill_color=[['white'], pic, revc, travc, latc,spdc],
    align='left', font=dict(color='black', size=14)
    ))
])
fig.update_layout(font=dict(family="ibm plex sans",size=18,),width=1200,height=800)
fig.update_traces(cells_height=40, selector=dict(type='table'))
#fig.write_image("D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\Images\\Nicole's figures\\Chrimson\\Table of PI numbers.svg")
fig.show()

C:\Users\user\AppData\Local\Temp\ipykernel_53392\2502285022.py:23: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

df10 = df1.copy()
df10.drop('Unnamed: 0', axis=1, inplace=True) 
df2=df10.copy()
df2.loc[len(df2)] = ['test', 1, 1,1,1,1]
df2.loc[len(df2)] = ['test', -1,-1,-1,-1,-1]

basegenotype = df2['Genotype'].unique()


def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())
for col in df2.columns[1:]:
    df2.loc[((df2[col]>1)), [col]] = 1
    df2.loc[((df2[col]<-1)), [col]] = -1
    df2[col] = min_max_scaling(df2[col])

rdbu = cm.get_cmap('RdBu_r', 100)
pic, revc, travc, latc,spdc=[],[],[],[],[]
for n in range(0,len(df2)):
    pic.append('rgb(' + str(rdbu(round(df2.iloc[n,1]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,1]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,1]*100))[2]*255)+')')
    revc.append('rgb(' + str(rdbu(round(df2.iloc[n,2]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,2]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,2]*100))[2]*255)+')')
    travc.append('rgb(' + str(rdbu(round(df2.iloc[n,3]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,3]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,3]*100))[2]*255)+')')
    latc.append('rgb(' + str(rdbu(round(df2.iloc[n,4]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,4]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,4]*100))[2]*255)+')')
    spdc.append('rgb(' + str(rdbu(round(df2.iloc[n,5]*100))[0]*255)+','+str(rdbu(round(df2.iloc[n,5]*100))[1]*255)+','+str(rdbu(round(df2.iloc[n,5]*100))[2]*255)+')')

pi, rev, trav, lat,spd=[],[],[],[],[]
for n in range(0,len(df1)):
    pi.append(str("{:.2f}".format(df1.iloc[n,1])))
    rev.append(str("{:.2f}".format(df1.iloc[n,2])))
    trav.append(str("{:.2f}".format(df1.iloc[n,3])))
    lat.append(str("{:.2f}".format(df1.iloc[n,4])))
    spd.append(str("{:.2f}".format(df1.iloc[n,5])))
    
root = "D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\osar_test\\ACR\\"
startdir = os.listdir(root)
baseDirectory=[]
basegenotype=[]
for n,sd in enumerate(startdir):
    dir1 = os.path.join(root, os.listdir(root)[n])
    baseDirectory.append(dir1)
    basegenotype.append(re.search(r'^.*?(?= x)', (os.path.split(dir1)[1])).group(0))
    

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Genotype</b>', '<b>Δ pi</b>','<b>Δ Reversal pi</b>', '<b>Δ Traversal pi</b>','<b>Δ Light Attraction index</b>','<b>Δ Speed Ratio</b>'],
    line_color='darkslategray', fill_color='white',
    align='center',font=dict(color='black', size=14)
  ),
  cells=dict(
    values=[basegenotype, pi, rev, trav, lat,spd],
    line_color=['darkslategray'],
    fill_color=[['white'], pic, revc, travc, latc,spdc],
    align='left', font=dict(color='black', size=14)
    ))
])
fig.update_layout(font=dict(family="ibm plex sans",size=18,),width=1200,height=800)
fig.update_traces(cells_height=40, selector=dict(type='table'))
#fig.write_image("D:\\Nicole Lee - HDD\\ACC Lab Dropbox\\ACC Lab\\Nicole Lee\\DATA\\Images\\Nicole's figures\\ACR\\Table of PI numbers.svg")
fig.show()

C:\Users\user\AppData\Local\Temp\ipykernel_53392\2141879721.py:22: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



ValueError: Unknown format code 'f' for object of type 'str'